# Skapar modell för hela description text


In [16]:
# Skapar modell för hela description text
import pandas as pd
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

# Step 1: Load the JSON file using Pandas and extract the text data
data = pd.read_json('Testfil_FINAL.json')
description_text = data['description_text']

# Step 2: Preprocess the text data
processed_data = [simple_preprocess(text) for text in description_text]

# Step 3: Train the Word2Vec model
model = Word2Vec(processed_data, min_count=1, vector_size=100)

# Step 4: Save the trained model
model.save('word2vec_model_desc_text')

# You can now use the trained model for various tasks, such as word similarity or keyword-in-context searches.


# Börja här

In [25]:
# skapar modell för enstaka meningar i description text
import pandas as pd
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
nltk.download('punkt')


# Step 1: Load the JSON file using Pandas and extract the text data
data = pd.read_json('Testfil_FINAL_2.json')
description_text = data['description_text']

# Step 2: Tokenize sentences in the text data
sentences = []
for text in description_text:
    text_sentences = sent_tokenize(text)
    sentences.extend(text_sentences)

# Step 3: Preprocess the text data
processed_data = [simple_preprocess(sentence) for sentence in sentences]

# Step 4: Train the Word2Vec model
model = Word2Vec(processed_data, min_count=1, vector_size=100)

# Step 5: Save the trained model
model.save('word2vec_model_desc_sentence')

# You can now use the trained model for various tasks, such as word similarity or keyword-in-context searches.


[nltk_data] Downloading package punkt to /Users/home/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Alternativ kod som tar bort stopwords i processed_data


In [11]:
import pandas as pd
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

# Step 1: Load the JSON file using Pandas and extract the text data
data = pd.read_json('Testfil_FINAL.json')
description_text = data['description_text']

# Step 2: Tokenize sentences in the text data
sentences = []
for text in description_text:
    text_sentences = sent_tokenize(text)
    sentences.extend(text_sentences)

# Step 3: Preprocess the text data and remove stopwords
stop_words = set(stopwords.words('swedish'))  # Load Swedish stopwords from NLTK

processed_data = []
for sentence in sentences:
    # Tokenize and filter out stopwords
    tokens = simple_preprocess(sentence)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    processed_data.append(filtered_tokens)

'''# Step 4: Train the Word2Vec model
model = Word2Vec(processed_data, min_count=1, vector_size=100)

# Step 5: Save the trained model
model.save('word2vec_model_desc_sentence')

# You can now use the trained model for various tasks, such as word similarity or keyword-in-context searches.'''


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


"# Step 4: Train the Word2Vec model\nmodel = Word2Vec(processed_data, min_count=1, vector_size=100)\n\n# Step 5: Save the trained model\nmodel.save('word2vec_model_desc_sentence')\n\n# You can now use the trained model for various tasks, such as word similarity or keyword-in-context searches."

# Söker igenom alla meningar och utan ord prio

In [2]:
# Load the saved Word2Vec model
model = Word2Vec.load('word2vec_model_desc_sentence')

# Define a target sentence for similarity comparison
target_sentence = "Du är diplomatisk, har god analytisk förmåga och kan enkelt röra dig mellan detaljer och helheter."

# Preprocess the target sentence
processed_target = simple_preprocess(target_sentence)

print(processed_target)

# Find similar sentences
similar_sentences = []
for sentence in processed_data:
    if len(sentence) > 0:
        similarity = model.wv.n_similarity(processed_target, sentence)
        similar_sentences.append((sentence, similarity))

# Sort the similar sentences based on similarity score
similar_sentences = sorted(similar_sentences, key=lambda x: x[1], reverse=True)

# Print the top 5 similar sentences
for sentence, similarity in similar_sentences[:10]:
    original_sentence = ' '.join(sentence)
    print(f"Similarity: {similarity:.2f} Sentence: {original_sentence}")


['du', 'är', 'diplomatisk', 'har', 'god', 'analytisk', 'förmåga', 'och', 'kan', 'enkelt', 'röra', 'dig', 'mellan', 'detaljer', 'och', 'helheter']
Similarity: 1.00 Sentence: du är diplomatisk har god analytisk förmåga och kan enkelt röra dig mellan detaljer och helheter
Similarity: 0.94 Sentence: du är bra på problemlösning och har god analytisk förmåga
Similarity: 0.94 Sentence: du är analytisk och har en god förmåga att prioritera
Similarity: 0.94 Sentence: god kommunikativ och social förmåga samt ordning och reda är starka kort hos dig
Similarity: 0.94 Sentence: du är serviceinriktad noggrann ansvarskännande och har god förmåga att kommunicera med din omgivning
Similarity: 0.94 Sentence: du är strukturerad självgående och har förmåga att prioritera
Similarity: 0.94 Sentence: du är driven och ansvarstagande och har god kommunikativ förmåga
Similarity: 0.93 Sentence: är du drivande och strukturerad och har förmåga att samarbeta
Similarity: 0.93 Sentence: självfallet är du ansvarstagand

# PRIORITERAR ETT ORD FÖR VEKTORN SÅ ALLA MENINGAR UTAN ORDET EXKLUDERAS FRÅN SIMILARITETEN

In [3]:
# Load the saved Word2Vec model
model = Word2Vec.load('word2vec_model_desc_sentence')

# Define a target sentence for similarity comparison
target_sentence = "Du är diplomatisk, har god analytisk förmåga och kan enkelt röra dig mellan detaljer och helheter."

# Preprocess the target sentence
processed_target = simple_preprocess(target_sentence)

print(processed_target)

# Find similar sentences
similar_sentences = []
for sentence in processed_data:
    if len(sentence) > 0 and 'analytisk' in sentence:  # Check if 'priority_word' is present in the sentence
        similarity = model.wv.n_similarity(processed_target, sentence)
        similar_sentences.append((sentence, similarity))

# Sort the similar sentences based on similarity score
similar_sentences = sorted(similar_sentences, key=lambda x: x[1], reverse=True)

# Print the top 5 similar sentences
for sentence, similarity in similar_sentences[:10]:
    original_sentence = ' '.join(sentence)
    print(f"Similarity: {similarity:.2f} Sentence: {original_sentence}")


['du', 'är', 'diplomatisk', 'har', 'god', 'analytisk', 'förmåga', 'och', 'kan', 'enkelt', 'röra', 'dig', 'mellan', 'detaljer', 'och', 'helheter']
Similarity: 1.00 Sentence: du är diplomatisk har god analytisk förmåga och kan enkelt röra dig mellan detaljer och helheter
Similarity: 0.94 Sentence: du är bra på problemlösning och har god analytisk förmåga
Similarity: 0.94 Sentence: du är analytisk och har en god förmåga att prioritera
Similarity: 0.93 Sentence: vad vi vill att du har med dig har god analytisk kapacitet till helhetstänk har god förmåga att uttrycka dig väl tal och skrift har en god social kompetens har bra förmåga att arbeta såväl grupp som självständigt sätter kunden fokus och ge dem god service är serviceinriktad stresstålig noggrann strukturerad och ansvarsfull meriterande är om du har tidigare erfarenhet av oas äga ik collekt melos softronic är ett företag med såväl bredd som djup
Similarity: 0.92 Sentence: som person är du strukturerad har god analytisk förmåga och fö

# Topic analys på alla rader i dataset. Inte relevant då den inte tar bort stoppord och inte filtrerar på meningar med dåliga ord.

In [ ]:
from gensim import corpora, models

# Preprocess the sentences
#processed_data = [...]  # Your preprocessed sentences

# Create a dictionary from the preprocessed data
dictionary = corpora.Dictionary(processed_data)

# Create a corpus (bag of words representation) using the dictionary
corpus = [dictionary.doc2bow(sentence) for sentence in processed_data]

# Train the LDA model
num_topics = 5  # Specify the number of topics
lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

# Print the topics and their top keywords
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

# Get the topic distribution for each sentence in the dataset
topic_distributions = []
for sentence in processed_data:
    bow = dictionary.doc2bow(sentence)
    topic_distribution = lda_model.get_document_topics(bow)
    topic_distributions.append(topic_distribution)

# Visualize the dominant topic for each sentence
for i, topic_dist in enumerate(topic_distributions):
    dominant_topic = max(topic_dist, key=lambda x: x[1])
    print(f"Sentence {i+1}: Dominant Topic - {dominant_topic[0]}")


# Genererar mening utifrån best guess på efterkommande ord



In [11]:
# Load the saved Word2Vec model
model = Word2Vec.load('word2vec_model_desc_sentence')

# Define a target sentence for similarity comparison
target_sentence = "Du är diplomatisk, har god analytisk förmåga och kan enkelt röra dig mellan detaljer och helheter."

# Preprocess the target sentence
processed_target = simple_preprocess(target_sentence)

# Initialize an empty list to store the generated sentence
generated_sentence = []

# Iterate over each word in the target sentence
for target_word in processed_target:
    # Check if the target word is in the vocabulary of the Word2Vec model
    if target_word in model.wv:
        # Get the most similar word to the target word
        similar_word = model.wv.most_similar(positive=[target_word], topn=1)[0][0]
        generated_sentence.append(similar_word)

# Print the generated sentence
generated_sentence = ' '.join(generated_sentence)
print(f"Generated Sentence: {generated_sentence}")


Generated Sentence: man vara asd ha hög pedagogisk förmågan samt måste ladda befinna sig utgångspunkten helheten samt helheten


In [4]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
nltk.download('stopwords')

# Load the saved Word2Vec model
model = Word2Vec.load('word2vec_model_desc_sentence')

# Define a target sentence for similarity comparison
target_sentence = "Du är diplomatisk, har god analytisk förmåga och kan enkelt röra dig mellan detaljer och helheter."

# Preprocess the target sentence
stop_words = set(stopwords.words('swedish'))  # Load Swedish stop words from NLTK
processed_target = [word for word in simple_preprocess(target_sentence) if word not in stop_words]

print(processed_target)

# Find similar sentences
similar_sentences = []
for sentence in processed_data:
    # Remove stop words from the sentence
    processed_sentence = [word for word in sentence if word not in stop_words]
    if len(sentence) > 0 and 'analytisk' in sentence:  # Check if 'priority_word' is present in the sentence
        similarity = model.wv.n_similarity(processed_target, processed_sentence)
        similar_sentences.append((sentence, similarity))



# Sort the similar sentences based on similarity score
similar_sentences = sorted(similar_sentences, key=lambda x: x[1], reverse=True)

# Print the top 5 similar sentences
for sentence, similarity in similar_sentences[:10]:
    original_sentence = ' '.join(sentence)
    print(f"Similarity: {similarity:.2f} Sentence: {original_sentence}")


[nltk_data] Downloading package stopwords to /Users/home/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['diplomatisk', 'god', 'analytisk', 'förmåga', 'enkelt', 'röra', 'detaljer', 'helheter']
Similarity: 1.00 Sentence: du är diplomatisk har god analytisk förmåga och kan enkelt röra dig mellan detaljer och helheter
Similarity: 0.97 Sentence: du är analytisk och har en god förmåga att prioritera
Similarity: 0.96 Sentence: god analytisk förmåga
Similarity: 0.96 Sentence: vidare har du god analytisk förmåga
Similarity: 0.95 Sentence: du är bra på problemlösning och har god analytisk förmåga
Similarity: 0.94 Sentence: din yrkesroll är du serviceminded noggrann strukturerad samt med en god analytisk förmåga
Similarity: 0.93 Sentence: vidare drivs du av problemlösning och har en god analytisk förmåga
Similarity: 0.92 Sentence: du har en mycket god analytisk förmåga och är pedagogisk uthållig och har hög integritet
Similarity: 0.92 Sentence: som person är du strukturerad har god analytisk förmåga och förståelse för komplexa flöden och beroendesamband
Similarity: 0.92 Sentence: du bör ha god ana

# TOPICS UTIFRÅN MENINGAR SOM INNEHÅLLER TARGET WORD. FUNGERAR SÅDÄR DÅ DEN BEHÖVER EN TARGET SENTENCE

In [79]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from gensim import corpora, models
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download('stopwords')

# Load the saved Word2Vec model
model = Word2Vec.load('word2vec_model_desc_sentence')

# Define a target sentence for similarity comparison
target_sentence = "Du är diplomatisk, har god analytisk förmåga och kan enkelt röra dig mellan detaljer och helheter."

# Preprocess the target sentence
stop_words = set(stopwords.words('swedish'))  # Load Swedish stop words from NLTK
processed_target = [word for word in simple_preprocess(target_sentence) if word not in stop_words]

# Tokenize sentences in the text data
sentences = []
for text in description_text:
    text_sentences = sent_tokenize(text)
    sentences.extend(text_sentences)

# Preprocess the sentences and remove stop words
processed_data = []
for sentence in sentences:
    processed_sentence = [word for word in simple_preprocess(sentence) if word not in stop_words]
    processed_data.append(processed_sentence)

# Filter the sentences to consider only those containing 'analytisk'
analytisk_sentences = [sentence for sentence in processed_data if 'analytisk' in sentence]

# Train the Word2Vec model
model = Word2Vec(analytisk_sentences, min_count=1, vector_size=100)

# Find similar sentences
similar_sentences = []
for sentence in analytisk_sentences:
    if len(sentence) > 0:
        similarity = model.wv.n_similarity(processed_target, sentence)
        similar_sentences.append((sentence, similarity))

# Sort the similar sentences based on similarity score
similar_sentences = sorted(similar_sentences, key=lambda x: x[1], reverse=True)

# Print the top 5 similar sentences
for sentence, similarity in similar_sentences[:10]:
    original_sentence = ' '.join(sentence)
    print(f"Similarity: {similarity:.2f} Sentence: {original_sentence}")

# Remove stop words from the preprocessed data
processed_data_no_stopwords = [[word for word in sentence if word not in stop_words] for sentence in analytisk_sentences]

# Create a dictionary from the preprocessed data
dictionary = corpora.Dictionary(processed_data_no_stopwords)

# Create a corpus (bag of words representation) using the dictionary
corpus = [dictionary.doc2bow(sentence) for sentence in processed_data_no_stopwords]

# Train the LDA model
num_topics = 3  # Specify the number of topics
lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

# Get the topic distribution for each sentence in the dataset
topic_distributions = []
for sentence in processed_data_no_stopwords:
    bow = dictionary.doc2bow(sentence)
    topic_distribution = lda_model.get_document_topics(bow)
    topic_distributions.append(topic_distribution)

# Visualize the dominant topic for each sentence
for i, topic_dist in enumerate(topic_distributions):
    dominant_topic = max(topic_dist, key=lambda x: x[1])
    print(f"Sentence {i+1}: Dominant Topic - {dominant_topic[0]}")




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Similarity: 1.00 Sentence: diplomatisk god analytisk förmåga enkelt röra detaljer helheter
Similarity: 1.00 Sentence: god analytisk förmåga
Similarity: 1.00 Sentence: analytisk god förmåga prioritera
Similarity: 1.00 Sentence: vidare god analytisk förmåga
Similarity: 1.00 Sentence: vidare förmåga hantera stora mängder information kombination god analytisk förmåga
Similarity: 1.00 Sentence: vidare förmåga hantera stora mängder information kombination god analytisk förmåga
Similarity: 1.00 Sentence: person positiv attityd arbete god analytisk förmåga
Similarity: 1.00 Sentence: person ser god analytisk förmåga framgångsrikt analysera bedöma it händelser
Similarity: 1.00 Sentence: tekniska miljön komplex kräver god analytisk förmåga samt kunna beskriva tekniska problem slutanvändaren vardagligt sätt
Similarity: 1.00 Sentence: tycker lösa problem god analytisk förmåga
Sentence 1: Dominant Topic - 0
Sentence 2: Dominant Topic - 1
Sentence 3: Dominant Topic - 2
Sentence 4: Dominant Topic - 2


# Visar ord i topics och vilken topic som är vanligast i dataset

In [80]:

# Print the topics and their top keywords
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)



# Count the frequencies of dominant topics
dominant_topic_freq = {}
for topic_dist in topic_distributions:
    dominant_topic = max(topic_dist, key=lambda x: x[1])[0]
    dominant_topic_freq[dominant_topic] = dominant_topic_freq.get(dominant_topic, 0) + 1

# Find the most common dominant topic
most_common_topic = max(dominant_topic_freq, key=dominant_topic_freq.get)
frequency = dominant_topic_freq[most_common_topic]

print(f"Most Common Dominant Topic: {most_common_topic} (Frequency: {frequency})")

# ...

(0, '0.013*"kommer" + 0.009*"erfarenhet" + 0.007*"arbeta" + 0.006*"söker" + 0.006*"tjänsten" + 0.006*"work" + 0.006*"academic" + 0.005*"kunskaper" + 0.005*"it" + 0.005*"vill"')
(1, '0.016*"erfarenhet" + 0.011*"arbeta" + 0.010*"it" + 0.010*"kommer" + 0.008*"analytisk" + 0.007*"samt" + 0.007*"söker" + 0.007*"god" + 0.006*"förmåga" + 0.006*"tjänsten"')
(2, '0.029*"analytisk" + 0.016*"förmåga" + 0.014*"god" + 0.009*"person" + 0.008*"samt" + 0.008*"ser" + 0.007*"strukturerad" + 0.007*"arbeta" + 0.006*"både" + 0.006*"söker"')
Most Common Dominant Topic: 2 (Frequency: 218)


# (VIKTIG) Tar tre första orden i mest dominant topic (manuellt) och skickar in det som en sträng för att visa hur det kan se ut i en mening i dataset.


In [41]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


# Load the saved Word2Vec model
model = Word2Vec.load('word2vec_model_desc_sentence')

# Define a target sentence for similarity comparison
target_sentence = "rekryterande nämsta tillsammans"

# Preprocess the target sentence
stop_words = set(stopwords.words('swedish'))  # Load Swedish stop words from NLTK
processed_target = [word for word in simple_preprocess(target_sentence) if word not in stop_words]

print(processed_target)

# Find similar sentences
similar_sentences = []
for sentence in processed_data:
    # Remove stop words from the sentence
    processed_sentence = [word for word in sentence if word not in stop_words]
    if len(sentence) > 0 and 'chef' in sentence:  # Check if 'priority_word' is present in the sentence
        similarity = model.wv.n_similarity(processed_target, processed_sentence)
        similar_sentences.append((sentence, similarity))



# Sort the similar sentences based on similarity score
similar_sentences = sorted(similar_sentences, key=lambda x: x[1], reverse=True)

# Print the top 5 similar sentences
for sentence, similarity in similar_sentences[:20]:
    original_sentence = ' '.join(sentence)
    print(f"Similarity: {similarity:.2f} Sentence: {original_sentence}")


['rekryterande', 'nämsta', 'tillsammans']
Similarity: 0.74 Sentence: tillsammans med din chef sätter ni sedan kontinuerligt mål arbetet och utbildningsmål
Similarity: 0.69 Sentence: hur du utvecklas framöver och vad du vill jobba med framtiden bestämmer du tillsammans med din chef och ni gör tillsammans en planering för din framtida karriär
Similarity: 0.65 Sentence: jag som chef tillsammans med mina teamledare arbetar för att du som medarbetare skall utvecklas din vardag
Similarity: 0.63 Sentence: på semcon får du tillsammans med ett sammansvetsat team och en lyhörd chef möjligheten att utveckla samtidens och framtidens autonoma system med våra kundbolag lokalt regionalt och globalt
Similarity: 0.62 Sentence: du får vara med att forma din tjänst tillsammans med it chef och dina kollegor
Similarity: 0.62 Sentence: peter engberg som är chef på it avdelningen förklarar att dem söker en social kollega som anpassar sitt och fungerar bra tillsammans med teamet och andra delar av tieto
Simil